# 🧬 Championship DNA Analysis - Find What Actually Wins

**Forget "Zero-RB" and "Hero-RB" labels. Let's find what WINNING teams actually look like!**

This notebook analyzes thousands of simulated drafts to identify the common patterns in top-performing teams.
Instead of following rigid strategies, we'll discover the "DNA" of championship rosters.

In [1]:
# Setup
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Setup complete")

✅ Setup complete


## 🎯 Step 1: Set Your Draft Position

In [2]:
# CHANGE THIS TO YOUR DRAFT POSITION!
MY_DRAFT_POSITION = 5  # Your pick number (1-14)
N_SIMULATIONS = 2000   # More = better patterns (but slower)
TOP_PERCENT = 10       # Analyze top 10% as "winners"

print(f"📍 Your draft position: Pick #{MY_DRAFT_POSITION}")
print(f"🎲 Running {N_SIMULATIONS} simulations")
print(f"🏆 Analyzing top {TOP_PERCENT}% as championship teams")

📍 Your draft position: Pick #5
🎲 Running 2000 simulations
🏆 Analyzing top 10% as championship teams


## 🚀 Step 2: Run Simulations & Find Winners

In [ ]:
from src.monte_carlo import DraftSimulator

print("🎲 Running simulations...")
sim = DraftSimulator()

# Run simulations
result = sim.simulator.run_simulations(
    my_team_idx=MY_DRAFT_POSITION - 1,
    strategy_name='balanced',  # Use balanced to get variety
    n_sims=N_SIMULATIONS,
    base_seed=42
)

# Extract rosters
all_rosters = result.get('all_results', [])
print(f"✅ Generated {len(all_rosters)} rosters")

# Find the winners (top X%)
values = [r['roster_value'] for r in all_rosters]
cutoff = np.percentile(values, 100 - TOP_PERCENT)
winners = [r for r in all_rosters if r['roster_value'] >= cutoff]
losers = [r for r in all_rosters if r['roster_value'] < np.percentile(values, 25)]

print(f"\n🏆 Found {len(winners)} winning rosters (value >= {cutoff:.1f})")
print(f"📊 Average winner: {np.mean([r['roster_value'] for r in winners]):.1f} points")
print(f"📊 Average team: {np.mean(values):.1f} points")

🎲 Running simulations...
Created envelope data from projections (±20%)


## 🧬 Step 3: Discover Championship DNA

In [ ]:
# Analyze position composition
def analyze_roster_composition(rosters, label="Teams"):
    position_counts = defaultdict(list)
    
    for roster in rosters:
        pos_count = defaultdict(int)
        for player in roster['roster']:
            pos_count[player['pos']] += 1
        
        for pos, count in pos_count.items():
            position_counts[pos].append(count)
    
    # Create DataFrame for easy analysis
    comp_data = {}
    for pos in ['QB', 'RB', 'WR', 'TE', 'K', 'DST']:
        if pos in position_counts:
            counts = position_counts[pos]
            comp_data[pos] = {
                'avg': np.mean(counts),
                'mode': max(set(counts), key=counts.count),
                'min': min(counts),
                'max': max(counts)
            }
    
    return pd.DataFrame(comp_data).T

# Compare winners vs average
winner_comp = analyze_roster_composition(winners, "Winners")
all_comp = analyze_roster_composition(all_rosters, "All Teams")

print("🏆 WINNING ROSTER COMPOSITION:")
print("="*50)
print(winner_comp[['avg', 'mode']].round(1))

print("\n📊 DIFFERENCE FROM AVERAGE:")
print("="*50)
diff = winner_comp['avg'] - all_comp['avg']
for pos in ['QB', 'RB', 'WR', 'TE']:
    if pos in diff.index:
        sign = "+" if diff[pos] > 0 else ""
        print(f"{pos}: {sign}{diff[pos]:.2f} vs average")

## 🎯 Step 4: Early Round Patterns

In [ ]:
# Analyze draft start patterns
def get_early_patterns(rosters, n_rounds=3):
    patterns = []
    for roster in rosters:
        if 'position_sequence' in roster and len(roster['position_sequence']) >= n_rounds:
            pattern = tuple(roster['position_sequence'][:n_rounds])
            patterns.append(pattern)
    return patterns

# Get patterns for winners
winner_patterns = get_early_patterns(winners, 3)
all_patterns = get_early_patterns(all_rosters, 3)

# Count and compare
winner_counts = Counter(winner_patterns)
all_counts = Counter(all_patterns)

print("🔥 TOP 5 WINNING DRAFT STARTS (First 3 Rounds):")
print("="*60)

for i, (pattern, count) in enumerate(winner_counts.most_common(5), 1):
    win_rate = (count / len(winner_patterns)) * 100
    overall_rate = (all_counts.get(pattern, 0) / len(all_patterns)) * 100
    lift = win_rate / overall_rate if overall_rate > 0 else 0
    
    pattern_str = ' → '.join(pattern)
    print(f"{i}. {pattern_str:20} | {win_rate:5.1f}% of winners")
    print(f"   {'':20} | {overall_rate:5.1f}% overall ({lift:.1f}x better)")
    print()

## 📊 Step 5: Value Tier Analysis

In [ ]:
# Analyze projection values by position
def analyze_value_tiers(rosters, label="Teams"):
    position_values = defaultdict(list)
    
    for roster in rosters:
        # Group players by position and sort by value
        pos_players = defaultdict(list)
        for player in roster['roster']:
            pos_players[player['pos']].append(player['proj'])
        
        # Sort each position's players
        for pos, values in pos_players.items():
            values.sort(reverse=True)
            position_values[pos].extend(values)
    
    # Calculate tier thresholds
    tier_data = {}
    for pos in ['QB', 'RB', 'WR', 'TE']:
        if pos in position_values and position_values[pos]:
            values = position_values[pos]
            tier_data[pos] = {
                'Best': np.percentile(values, 90),
                'Good': np.percentile(values, 70),
                'Avg': np.percentile(values, 50),
                'Min': np.percentile(values, 25)
            }
    
    return pd.DataFrame(tier_data).T

winner_tiers = analyze_value_tiers(winners, "Winners")
all_tiers = analyze_value_tiers(all_rosters, "All")

print("💎 VALUE TIERS IN WINNING ROSTERS:")
print("="*60)
print(winner_tiers.round(0))

print("\n📈 WINNER ADVANTAGE BY POSITION:")
print("="*40)
for pos in ['QB', 'RB', 'WR', 'TE']:
    if pos in winner_tiers.index and pos in all_tiers.index:
        diff = winner_tiers.loc[pos, 'Best'] - all_tiers.loc[pos, 'Best']
        print(f"{pos}: +{diff:.0f} points at top end")

## 🎯 Step 6: Your Personal Draft Blueprint

In [ ]:
print("="*70)
print("🎯 YOUR CHAMPIONSHIP DRAFT BLUEPRINT")
print(f"   For Pick #{MY_DRAFT_POSITION} in a 14-Team League")
print("="*70)

# Generate specific recommendations
print("\n📋 ROSTER CONSTRUCTION TARGETS:")
print("-"*40)
for pos in ['QB', 'RB', 'WR', 'TE']:
    if pos in winner_comp.index:
        target = int(winner_comp.loc[pos, 'mode'])
        avg = winner_comp.loc[pos, 'avg']
        print(f"{pos:3}: Draft {target} total (winners avg {avg:.1f})")

print("\n🔥 RECOMMENDED DRAFT STARTS:")
print("-"*40)
for i, (pattern, count) in enumerate(winner_counts.most_common(3), 1):
    pattern_str = ' → '.join(pattern)
    win_rate = (count / len(winner_patterns)) * 100
    print(f"{i}. {pattern_str:20} ({win_rate:.0f}% win rate)")

print("\n💰 VALUE TARGETS BY POSITION:")
print("-"*40)
for pos in ['QB', 'RB', 'WR', 'TE']:
    if pos in winner_tiers.index:
        best = winner_tiers.loc[pos, 'Best']
        good = winner_tiers.loc[pos, 'Good']
        print(f"{pos}: Target {best:.0f}+ for elite, {good:.0f}+ for solid")

# Key insights
print("\n⚡ KEY INSIGHTS:")
print("-"*40)

insights = []

# RB analysis
rb_early = sum(1 for p in winner_patterns if p[0] == 'RB') / len(winner_patterns)
if rb_early > 0.5:
    insights.append(f"• {rb_early*100:.0f}% of winners start with RB - strongly consider it")

# WR analysis  
wr_count = winner_comp.loc['WR', 'avg'] if 'WR' in winner_comp.index else 0
if wr_count >= 3.5:
    insights.append(f"• Load up on WRs - winners average {wr_count:.1f}")

# TE analysis
te_early = sum(1 for p in winner_patterns if 'TE' in p[:3]) / len(winner_patterns)
if te_early > 0.25:
    insights.append(f"• {te_early*100:.0f}% grab elite TE early (rounds 1-3)")
elif te_early < 0.1:
    insights.append(f"• Only {te_early*100:.0f}% take TE early - wait on TE")

# QB analysis
qb_early = sum(1 for p in winner_patterns if 'QB' in p[:3]) / len(winner_patterns)
if qb_early < 0.15:
    insights.append(f"• Only {qb_early*100:.0f}% take QB in rounds 1-3 - wait!")

for insight in insights:
    print(insight)

print("\n" + "="*70)
print("✅ Blueprint complete! Draft based on PATTERNS, not LABELS.")
print("="*70)

## 📊 Bonus: Visualization

In [ ]:
# Create comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Championship DNA Analysis', fontsize=16, fontweight='bold')

# 1. Position composition comparison
ax = axes[0, 0]
positions = ['QB', 'RB', 'WR', 'TE']
winner_avgs = [winner_comp.loc[p, 'avg'] if p in winner_comp.index else 0 for p in positions]
all_avgs = [all_comp.loc[p, 'avg'] if p in all_comp.index else 0 for p in positions]

x = np.arange(len(positions))
width = 0.35
ax.bar(x - width/2, winner_avgs, width, label='Winners', color='gold')
ax.bar(x + width/2, all_avgs, width, label='Average', color='silver')
ax.set_xlabel('Position')
ax.set_ylabel('Average Count')
ax.set_title('Roster Composition: Winners vs Average')
ax.set_xticks(x)
ax.set_xticklabels(positions)
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Value distribution
ax = axes[0, 1]
winner_values = [r['roster_value'] for r in winners]
all_values = [r['roster_value'] for r in all_rosters]
ax.hist(all_values, bins=30, alpha=0.5, label='All Teams', color='blue')
ax.hist(winner_values, bins=20, alpha=0.7, label='Winners', color='gold')
ax.axvline(cutoff, color='red', linestyle='--', label=f'Top {TOP_PERCENT}% cutoff')
ax.set_xlabel('Roster Value')
ax.set_ylabel('Frequency')
ax.set_title('Roster Value Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Top draft patterns
ax = axes[1, 0]
top_5_patterns = winner_counts.most_common(5)
pattern_labels = [' → '.join(p[0]) for p in top_5_patterns]
pattern_counts_list = [p[1] for p in top_5_patterns]
pattern_pcts = [c/len(winner_patterns)*100 for c in pattern_counts_list]

ax.barh(range(len(pattern_labels)), pattern_pcts, color='teal')
ax.set_yticks(range(len(pattern_labels)))
ax.set_yticklabels(pattern_labels)
ax.set_xlabel('% of Winners')
ax.set_title('Top 5 Winning Draft Starts (First 3 Rounds)')
ax.grid(True, alpha=0.3, axis='x')

# 4. Position value tiers
ax = axes[1, 1]
positions = ['QB', 'RB', 'WR', 'TE']
colors = ['red', 'green', 'blue', 'orange']

for i, pos in enumerate(positions):
    if pos in winner_tiers.index:
        values = [winner_tiers.loc[pos, col] for col in ['Min', 'Avg', 'Good', 'Best']]
        ax.plot(values, label=pos, marker='o', linewidth=2, color=colors[i])

ax.set_xticks(range(4))
ax.set_xticklabels(['25th%', '50th%', '70th%', '90th%'])
ax.set_ylabel('Projected Points')
ax.set_title('Value Tiers in Winning Rosters')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Visualization complete!")

## 💾 Save Your Blueprint

In [ ]:
# Save results to file
import json
from datetime import datetime

blueprint = {
    'generated': datetime.now().isoformat(),
    'draft_position': MY_DRAFT_POSITION,
    'simulations': N_SIMULATIONS,
    'top_percent': TOP_PERCENT,
    'roster_targets': winner_comp[['mode']].to_dict()['mode'],
    'value_targets': winner_tiers[['Best', 'Good']].to_dict(),
    'top_patterns': [{
        'pattern': ' → '.join(p[0]),
        'win_rate': p[1]/len(winner_patterns)
    } for p in winner_counts.most_common(5)],
    'insights': insights
}

filename = f'championship_blueprint_pick{MY_DRAFT_POSITION}.json'
with open(filename, 'w') as f:
    json.dump(blueprint, f, indent=2, default=str)

print(f"✅ Saved blueprint to {filename}")
print("\n🏆 You're ready to draft like a champion!")
print("   Focus on the PATTERNS, not the LABELS.")